# Praxisübung: Empfehlungsdienst

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

In dieser Übung geht es darum ein Empfehlungssystem für Filme mittels kollaborativem Filtern zu entwickeln. Wir beschränken uns dabei auf modellfreie Verfahren, d.h. nutzerbasiertes und objektbasiertes kollaboratives Filtern.

Wir betrachten dazu den MovieLens 100k Datensatz, der insgesamt 100,000 Nutzerbewertungen von Kinofilmen enthält (Download hier https://files.grouplens.org/datasets/movielens/ml-100k.zip oder in ILIAS).

Dazu lesen wir zunächst die Daten ein und benennen die Spalten.

In [ ]:
df = pd.read_csv("Daten/ml-100k/u.data", sep="\t", header=None)
df.columns = ["user_id", "item_id", "rating", "timestamp"]

df

Eine Zeile entspricht einer Bewertung eines Nutzers für einen Film (an einem Zeitpunkt). Weitere Informationen zu den Filmen ist in der Datei `u.item` enthalten.

In [ ]:
df_items = pd.read_csv("Daten/ml-100k/u.item", sep="|", header=None, encoding_errors="ignore")
df_items.columns = ["movie id", "movie title", "release date", "video release date", "IMDb URL", "unknown", "Action", "Adventure", "Animation", 
                    "Children's", "Comedy", "Crime", "Documentary", "Drama", "Fantasy", "Film-Noir", "Horror", "Musical",
                    "Mystery", "Romance", "Sci-Fi", "Thriller", "War", "Western"]
df_items

Wir joinen nur die Titel und arbeiten dann mit `df` weiter.

In [ ]:
df = df.merge(df_items[["movie id", "movie title"]], left_on="item_id", right_on="movie id")
df

## ✏ Aufgabe 1
1. Geben Sie die Anzahl der Benutzer und die Anzahl der Filme aus.
2. Plotten Sie die Verteilung der Bewertungen.

In [ ]:
# TO DO

## ✏ Aufgabe 2

Welches sind die beliebtesten Filme?

In [ ]:
# TO DO

## ✏ Aufgabe 3

Welches sind die beliebtesten Filme, die mindestens 10 Bewertungen haben? Fügen Sie dazu einer Spalte `n_ratings` hinzu, die die Anzahl der Bewertungen für einen Film enthält und führen Sie danach die Aggregation aus.

In [ ]:
# TO DO

---
Als nächstes spalten wir die Daten in Trainings- und Testset. Das Training besteht beim kollaborativen Filtern nur aus dem Ermitteln der Distanzen zwischen einzelnen Nutzern oder Objekten. Die Distanzen hängen allerdings davon ab, welche Bewertungen im Trainingsdatensatz sind.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test = train_test_split(df, shuffle=True, random_state=42, test_size=0.25)

Als nächstes müssen wir aus den Bewertungen die (unvollständige) Interaktionsmatrix aufbauen, in der eine Zeile einen Nutzer und eine Spalte einen Film repräsentiert.

Die bekannten Elemente der Interaktionsmatrix sind in den ersten drei Spalten unseres DataFrames spezifiziert: `rating` enthält die Matrixeinträge und `user_id` bzw. `item_id` enthalten die zugehörigen Zeilen- bzw. Spaltenindizes.

Dies ist auch ein gängiges Speicherformat für sog. *dünnbesetzte* (sparse) Matrizen: Matrizen, bei denen nur wenige Elemente ungleich 0 sind können platzsparend gespeichert werden, indem nur die Nichtnull-Einträge gespeichert werden. Ein gängiges Format dafür ist das Koordinatenformat: hierzu wird eine Matrix beliebiger Dimension als drei (eindimensionale) Arrays gleicher Länge abgespeichert:
- Der erste Array enthält die Daten
- Der zweite Array enthält die Zeilenindizes
- Der dritte Array enthält die Spaltenindizes

Wir nutzen diese Funktionalität um unseren Datensatz zunächst schnell in eine dünnbesetzte Matrix `np.coo_matrix` (*COOrdinate matrix*) umzuwandeln und anschließend in eine "normale" Matrix (NumPy Array), auf der wir dann weiterarbeiten.

In [ ]:
from scipy.sparse import coo_matrix

n_rows = len(df.user_id.unique())
n_cols = len(df.item_id.unique())

row = X_train["user_id"] - 1
col = X_train["item_id"] - 1
data = X_train["rating"]
user_item_matrix_s = coo_matrix((data, (row, col)), shape=(n_rows, n_cols))
user_item_matrix = user_item_matrix_s.toarray()

Für das kollaborative Filtern brauchen wir eine Metrik, die Distanzen zwischen Benutzern (beim nutzerbasierten kollaborativen Filtern) bzw. Filmen (beim objektbasierten kollaborativen Filtern) quantifiziert. In der Vorlesung haben wir zwei Metriken kennengelernt:

1. Pearson-Korrelation: 

$$dist(x,y)=1-\frac{\sum_{i\in I_{x,y}} (r(x,i)-\bar{r}_x)(r(y,i)-\bar{r}_y)}{ \sqrt{\sum_{i\in I_{x,y}} (r(x,i)-\bar{r}_x)^2 \sum_{i\in I_{x,y}} (r(y,i)-\bar{r}_y)^2} }$$

2. Kosinus-Ähnlichkeit:

$$dist(x,y)=1-\frac{\sum_{i\in I_{x,y}} r(x,i)r(y,i)}{ \sqrt{\sum_{i\in I_{x,y}} r(x,i)^2 \sum_{i\in I_{x,y}} r(y,i)^2} }$$

Wobei $I_{x,y}$ diejenigen Indizes der beiden Vektoren (zwei Benutzer oder zwei Objekte) sind, für die bei *beiden* Vektoren die Einträge bekannt sind (wo also Bewertungen vorliegen).

## ✏ Aufgabe 4

Wie groß ist die minimale bzw. maximale Distanz von zwei Objekten bei diesen beiden Metriken?

In [ ]:
# Antwort

---
Wir entscheiden uns zunächst für die Pearson-Korrelation. Ein wichtiges Detail ist die Indexmenge $I_{x,y}$: Es wird nicht über *alle* Werte der Vektoren summiert, sondern nur über die Einträge, die bei beiden vorhanden sind. Da dies mit NumPy nicht so einfach zu realisieren ist, benutzen wir Pandas. Wir fassen dazu die Interaktionsmatrix als DataFrame auf und kennzeichnen nicht vorhandene Ratings als NaNs. Die Pandas Implementierung der Korrelation ignoriert diese dann bei der Berechnung.

In [ ]:
user_item_matrix = pd.DataFrame(user_item_matrix)
user_item_matrix = user_item_matrix.mask(user_item_matrix == 0)

## ✏ Aufgabe 5
Wir möchten **nutzerbasiertes** kollaboratives Filtern implementieren. Dafür berechnen wir zunächst die Distanzen zwischen allen Benutzern mittels Pearson-Korrelation. 
1. Welche Dimension hat die Matrix, die die Distanzen zwischen allen Nutzern enthält?
2. Schauen Sie sich die Pandas Methode `DataFrame.corr` an https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.corr.html, berechnen Sie damit die Distanzmatrix und speichern Sie sie in einer Variable `dist`. Achten Sie darauf, dass alle Distanzen positiv sind (es können sich manchmal Rundungsfehler einschleichen). Wie ist der Parameter `min_periods` zu interpretieren? Wählen Sie einen geeigneten Wert.

In [ ]:
# TO DO

---
Für eine Vorhersage geht man nun wie folgt vor:
1. Identifiziere die `k` nächsten Nachbarn eines Nutzers, wobei `k` ein Hyperparameter ist.
2. Wähle als Vorhersage den Mittelwert über die Bewertungen der `k` nächsten Nachbarn.

Wir benutzen die `NearestNeighbors` Klasse aus `scikit-learn`, um die `k` nächsten Nachbarn zu bestimmen.

In [ ]:
from sklearn.neighbors import NearestNeighbors

k = 30
knn = NearestNeighbors(n_neighbors=k, metric="precomputed")
knn.fit(dist)

Mit der folgenden Methode können Sie einzelne Bewertungen für Paare (`user_id`, `item_id`) vorhersagen.

In [ ]:
def predict_rating(user_id, item_id, user_item_matrix, dist, knn):
    neighbors = knn.kneighbors(dist[user_id-1].reshape(1,-1), return_distance=False)
    col = user_item_matrix.iloc[neighbors[0], item_id-1]
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", category=RuntimeWarning)
        pred_rating = col.mean()
    return pred_rating

## ✏ Aufgabe 6
1. Verstehen Sie, was die Methode macht? Wie unterscheidet Sie sich von der Vorhersagefunktion aus der Vorlesung (Folie 13)?
2. Mit `with warnings.catch_warnings()` werden Warnungen unterdrückt, falls der Mittelwert über einen Vektor berechnet wird, der nur aus `nan`s besteht (der Mittelwert ist in diesem Fall auch `nan`). Wann tritt dieser Fall auf?
3. Benutzen Sie die Funktion um eine Vorhersage für `user_id=196` und `item_id=242` zu machen (dies ist der erste Eintrag im Datensatz). Wie groß ist die Abweichung zu der tatsächlichen Bewertung?

In [ ]:
# TO DO

## ✏ Aufgabe 7
1. Machen Sie Vorhersagen auf dem gesamten Trainings- und Testdatensatz und speichern Sie diese in einer zusätzlichen Spalte `pred_rating`. Sie können dafür einfach über alle Zeilen iterieren (`DataFrame.iterrows()`) und die Funktion `pred_rating` ausführen.
2. Berechnen Sie die mittlere absolute Abweichung auf beiden Datensätzen.

In [ ]:
# TO DO

## ✏ Aufgabe 8
Bisher ging es um *Vorhersagen* von Bewertungen. Relevanter sind aber oft *Empfehlungen* (Hintergrund: bei Filmen, die dem Nutzer ohnehin nicht gefallen, ist die Vorhersagegenauigkeit nicht so wichtig). Als nächstes soll das System Empfehlungen machen können.

1. Schreiben Sie eine Funktion, die für jeden Benutzer die 5 Filme mit der besten vorhergesagten Bewertung empfiehlt.
2. Berechnen Sie die mittlere absolute Abweichung nur für die Empfehlungen.

In [ ]:
# TO DO

## ✏ Weitere Aufgaben 
Implementieren Sie als nächstes folgende Varianten des Systems und evaluieren Sie diese.
1. Benutzen Sie die Kosinus-Ähnlichkeit statt der Korrelation als Abstandsmaß. Tipp: Ersetzen Sie vorher alle `nan`s durch den Mittelwert pro User bevor Sie die Distanzmatrix berechnen.
2. Machen Sie statt nutzerbasiertem ein objektbasiertes kollaboratives Filtern. Starten Sie, indem Sie die Distanzmatrix für alle *Objekte* berechnen. 
3. Die Vorhersage für die Bewertung wurde in der Vorlesung etwas anders eingeführt: $$ \tilde{r}(x,s)=\bar{r}_x+\frac{1}{k} \sum_{x'\in kNN(x)} (r(x',s)-\bar{r}_{x'})$$ Modifizieren Sie das Notebook so, dass die Funktion $\tilde{r}$ für die Vorhersage benutzt wird. 
- Vergleich mit state of the art: http://www.mymedialite.net/examples/datasets.html


In [ ]:
# TO DO